In [1]:
import os, glob
import pandas as pd
import time
import math
import re
import numpy as np
from tqdm.notebook import tqdm as tqdm
import uuid 
from enum import Enum
from metaphone import doublemetaphone
import pytest
import unicodedata
import itertools

In [2]:
def combine_addr(MHSNUMB, MPREDIR, MSTNAME, MMODE):
    return re.sub('nan+', '', str(MHSNUMB)+' '+str(MPREDIR)+' '+str(MSTNAME)+' '+str(MMODE))

In [3]:
keywords=[" BANK ", " CORP", " LLC", " INC", " LTD", " HRS", "MGT", "CORPORATION",
          "PARTICIPATION", " TRUST", " TRUS", " OF ", " CO ", " LP"
          "UNIVERSITY", "COLLEGE", " CHURCH", "STATE", " CLUB",
          "BAPTISI", "EVANGELICAL", "METHODIST", "CATHOLIC", "PROPERTIES",
          "ASSOCIATIONS", "ASSOCIATES", " TOWN OF", "CITY OF", " ASSOCI", 
         "SOCIETY", "MAINTENANCE", "MAINTENANC", " COUNTY", "TELEPHONE",
         "ELECTRIC", "ENTERPRISES", "ENTERPRISE", "AUTHORITY", "HOMEOWNERS",
         "INTERNATIONAL", " MINISTRY", " OFFICE", "INVESTMENT", 
         " HOME", " MGMT", " PRESBYTERIAN", " INN", "ASSOCIATION",
          " STE ", " L L C", "ACCOUNTING", "MAINTEN", "PRODUCT", "MUTUAL", 
         "ESTATES", "PARTNER", " & SONS", "FINANCE", " TITLE", " L P", " FARM",
         "WIRELESS", "COMMUNICATION", " SERVICE", "BAR & GRILL", " DEPT",
         "DEPARTMENT", " CTR", " LOAN", "SPECIALTIES", "BRANDS", " UNITED"
         "CREDIT", " UNION", "CORPORATE", "TREASURER", " ADMIN", "UTILITIES", 
         "COMMERCIAL", " STORAGE", "FAMILY"]    

biz_word_drop=[" CORP", " LLC", " INC", " LTD", " HRS", "MGT", " OF ", " CO ", " LP", " MGMT"
               " STE ", " L L C", " L P"]

junior_keywords=[" JR ", " JR. ", " Jr ", " Jr. ", " jr ", 
                 " jr. ", " JR", " JR.", " Jr", " Jr.", " jr", 
                 " jr.",  ' II ', ' III ', ' IV ']

In [4]:
def initial_classification(x):
    wrd_srch = re.compile(r'"'+"\b"+('|'.join(keywords))+"\b")
    is_junior = re.compile(r'"'+"\b"+('|'.join(junior_keywords))+"\b")
    #this will isolate other legal organization by key words
    if wrd_srch.search(str(x)):
        return 0
    #this isolates the junion records to be treated seperately
    elif is_junior.search(str(x)):
        return 5
    #This section tests the properties of the string, including position of the comma
    elif type(x) == str:
        if len(x.split()) > 1:
            if len(x.split(',')[0].split()) == 1:
                return 1
            else:
                return 2
        else:
            return 3
    else:
        return 4


In [5]:
def series_compare(s1, s2):
    tmp= []
    for index, values in s1.items():
        #if ether field is labelled as another legal entity
        if (values == 0) | (s2[index] == 0):
            tmp.append(0)
        #this will capture all juniors and run them seperately. 
        #I am electing to split them here instead of slowing down the matching function later
        elif (values == 5) | (s2[index] == 5):
            tmp.append(3)
        #if the OWN1 field is listed as an owner or as an undecided not picked up by the word search
        elif (values == 1) | (values == 2):
            tmp.append(1)
        #if own1 is not a standard naming convention length and own 2 is Nan
        elif (values == 3) & (s2[index] == 4):
            tmp.append(0)
        #if own1 is not a standard naming convention and own 2 is not empty. 
        elif (values == 3) & (s2[index] != 4):
            tmp.append(1)
        #if both fields are empty
        elif (values == 4) & (s2[index] == 4):
            #changed from 0 to 2. No values in own fields yield individual results
            tmp.append(2)
        #if field 1 is empty and field 2 us now
        elif (values == 4) & (s2[index] != 4):
            tmp.append(-99999)
        else:
            tmp.append(10)
    return(pd.Series(tmp))

In [6]:
NameCleaner= ["TTEE", "DR.","MR.", "MS.", "MRS.", "CAPTAIN", "CPT.", "PROF", "REV." "COACH", "PROFESSOR", "REVEREND" ,"SIR", "LT.", "SGT.", "SR."]


abbr=pd.read_csv(os.path.dirname(os.getcwd())+'\\INPUTS\\Common_Name_Abbreviations.csv', low_memory=False, names=["Ab", "Full"])


NamesExpander={}
for i in range(len(abbr)):
    NamesExpander.update( {abbr["Ab"][i] : abbr["Full"][i]} )
    

def double_metaphone(value):
    return doublemetaphone(value)


def normalize_unicode_to_ascii(data):
    normal = unicodedata.normalize('NFKD', data).encode('ASCII', 'ignore')
    val = normal.decode("utf-8")
    val = val.lower()
    # remove special characters
    val = re.sub('[^A-Za-z0-9 ]+', ' ', val)
    # remove multiple spaces
    val = re.sub(' +', ' ', val)
    return val


def generate_combinations(name_tuple):
    coms = []
    coms.append((name_tuple))
    i = len(list(name_tuple)) - 1
    if i > 1:
        coms.extend(itertools.combinations(name_tuple,i))

    return coms


def generate_normalized_name(name_tuple):
    name_arr = list(name_tuple)
    name_arr.sort()
    name_str = ''.join(name_arr)
    return name_str.lower()

def add_combinations_to_directory(comb_tuples, person_id):
    for comb in comb_tuples:
        concat_name = generate_normalized_name(comb)
        metaphone_tuple = doublemetaphone(concat_name)
        if metaphone_tuple[0] in __lookup_dict[0]:
            __lookup_dict[0][metaphone_tuple[0]].append(person_id)

        else:
            __lookup_dict[0][metaphone_tuple[0]] = [person_id]
        if metaphone_tuple[1] in __lookup_dict[1]:
            __lookup_dict[1][metaphone_tuple[1]].append(person_id)
        else:
            __lookup_dict[1][metaphone_tuple[1]] = [person_id]

def add_person_to_lookup_directory(person_id, name_tuple):
    add_combinations_to_directory(name_tuple, person_id)

def create_meta_names(x, id):
    add_person_to_lookup_directory(id, x)

In [7]:
def common_members(a, b):
    a_set = set(a)
    b_set = set(b)
    if len (a_set.intersection(b_set)) > 0:
        return(a_set.intersection(b_set))
    
def comb_own(x, y):
    global table
    if str(y)!= 'nan':
        owner = str(x)+''+str(y)
        return owner
    else:
        return str(x)

In [8]:
def name_prep():
    global table
    table['Owners'] = table['Owners'].str.upper()
    for clean in NameCleaner:
        table['Owners']=table['Owners'].str.replace(clean ,'')  
    for expand in NamesExpander:
        table['Owners'] = table['Owners'].str.replace(expand, NamesExpander[expand])
    tmp = re.compile(r"\b[a-zA-Z]\b")
    table['Owners'] =  table['Owners'].apply(lambda x: re.sub(tmp, "", x))
    table['Owners'] = table['Owners'].apply(lambda x: (normalize_unicode_to_ascii(x)).strip())
    table['Simple_Owners'] = table['Owners']
    table['Owners'] = tuple(table['Owners'].str.split(" "))
    table['Owners'] = table['Owners'].apply(lambda x: generate_combinations(x))

In [9]:
def match_name(combinations):
    match_list = []
    for comb_tuple in combinations:
        concat_name = generate_normalized_name(comb_tuple)
        metaphone_tuple = doublemetaphone(concat_name)
        if metaphone_tuple[0] in __lookup_dict[0]:
            match_list.append((concat_name, __lookup_dict[0][metaphone_tuple[0]]))
    return match_list


def update_match(x):
    global table
    a = table["Owners"].loc[table['Unq_ID'] == x].index.values
    p_list=[]
    for i in a:
        if i not in p_list:
            uid = uuid.uuid4()
            matches = match_name(table['Owners'][i])
            tmp = []
            for m in matches:
                y = common_members(a, m[1])
                if y != None:
                    for z in y:
                        if z not in tmp:
                            tmp.append(z)
            for g in tmp:
                table['Unq_ID'].iloc._setitem_with_indexer(g, str(uid))
            p_list.extend(tmp)
        else:
            continue

In [10]:
table=pd.read_csv(os.path.dirname(os.getcwd())+'\\OUTPUTS\\INTERMEDIATE\\temp.csv', low_memory=False)
table['Unq_ID'] = np.nan
table['Unq_ID']= table['Unq_ID'].astype('string')
table['comb_addr'] = table.apply(lambda x: combine_addr(x['MHSNUMB'], x['MPREDIR'], x['MSTNAME'], x['MMODE']), axis=1)
table['place_id'] = np.arange(len(table))
table['Owners'] = table.apply(lambda x: comb_own(str(x['OWN1']), str(x['OWN2'])), axis=1)
name_prep()

<ipython-input-8-43c0ca3f215d>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  table['Owners']=table['Owners'].str.replace(clean ,'')
<ipython-input-8-43c0ca3f215d>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  table['Owners'] = table['Owners'].str.replace(expand, NamesExpander[expand])
C:\Users\harri\Anaconda3\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


In [11]:
tmp_class1 = table["OWN1"].apply(initial_classification) 
tmp_class2 = table["OWN2"].apply(initial_classification) 

In [12]:
table['initial_class']=series_compare(tmp_class1, tmp_class2)

In [13]:
other=table.loc[table['initial_class']==0]
family=table.loc[table['initial_class']==1]
nans=table.loc[table['initial_class']==2]
jrs=table.loc[table['initial_class']==3]

In [14]:
#other
def iso_biz(col1, col2):
    wrd_srch = re.compile(r'"'+"\b"+('|'.join(keywords))+"\b")
    if wrd_srch.search(str(col1)):
        if wrd_srch.search(str(col2)):
            return str(col1)+' '+str(col2)
        else:
            return str(col1)
    elif wrd_srch.search(str(col2)):
        return str(col2)
    else:
        return(str(col1))


In [15]:
keywords.remove(" BANK ")
other['Owners'] = other.apply(lambda x: iso_biz(x["OWN1"], x['OWN2']), axis = 1) 

<ipython-input-15-37d700c9a391>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  other['Owners'] = other.apply(lambda x: iso_biz(x["OWN1"], x['OWN2']), axis = 1)


In [16]:
def other_prep():
    global other
    other['Owners'] = other['Owners'].str.upper()
    for clean in NameCleaner:
        other['Owners']=other['Owners'].str.replace(clean ,'')  
    for expand in NamesExpander:
        other['Owners'] = other['Owners'].str.replace(expand, NamesExpander[expand])
    for component in biz_word_drop:
        other['Owners']=other['Owners'].str.replace(component ,'')  
    tmp = re.compile(r"\b[a-zA-Z]\b")
    other['Owners'] =  other['Owners'].apply(lambda x: re.sub(tmp, "", x))
    other['Owners'] = other['Owners'].apply(lambda x: (normalize_unicode_to_ascii(x)).strip())
    other['Owners'] = tuple(other['Owners'].str.split(" "))
    other['Owners'] = other['Owners'].apply(lambda x: generate_combinations(x))

In [17]:
other_prep()

<ipython-input-16-f8cbb7cca060>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  other['Owners'] = other['Owners'].str.upper()
<ipython-input-16-f8cbb7cca060>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  other['Owners']=other['Owners'].str.replace(clean ,'')
<ipython-input-16-f8cbb7cca060>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  other['Owners']=other['Owners'].str.replace(clean ,'')
<ipython-input-16-f8cbb7cca060>:7: FutureWarning: The default v

In [18]:
__lookup_dict = ({}, {})

other.apply(lambda x: create_meta_names(x['Owners'], x['place_id']), axis=1)

17        None
18        None
20        None
21        None
22        None
          ... 
434499    None
434504    None
434505    None
434506    None
434507    None
Length: 85113, dtype: object

In [52]:
def update_match(meta_lists, unqid):
    global other
    if pd.isnull(unqid):
        uid = uuid.uuid4()
        matches = match_name(meta_lists)
        tmp = matches[0][1]
        for g in tmp:
            other.at[g, 'Unq_ID'] = str(uid)
    else:
        pass

In [53]:
#tqdm.pandas()
#other.progress_apply(lambda x: update_match(x['Owners'], x['Unq_ID']), axis = 1)
other.apply(lambda x: update_match(x['Owners'], x['Unq_ID']), axis = 1)

17        None
18        None
20        None
21        None
22        None
          ... 
434499    None
434504    None
434505    None
434506    None
434507    None
Length: 85113, dtype: object

In [54]:
#family

def update_match(x):
    global family
    a = family["Owners"].loc[family['Unq_ID'] == x].index.values
    p_list=[]
    for i in a:
        if i not in p_list:
            uid = uuid.uuid4()
            matches = match_name(family['Owners'][i])
            tmp = []
            for m in matches:
                y = common_members(a, m[1])
                if y != None:
                    for z in y:
                        if z not in tmp:
                            tmp.append(z)
            for g in tmp:
                family.at[g, 'Unq_ID'] = str(uid)
            p_list.extend(tmp)
        else:
            continue



__lookup_dict = ({}, {})
    


family.apply(lambda x: create_meta_names(x['Owners'], x['place_id']), axis=1)


uuid_dict = {key: str(uuid.uuid4()) for key in family['comb_addr'].unique()}
family['Unq_ID'] = family.comb_addr.map(uuid_dict)

x = family['Unq_ID'].unique()
f = lambda x: update_match(x)
#[f(xi) for xi in tqdm(x)]
[f(xi) for xi in x]

KeyboardInterrupt: 

In [22]:
#nans

uuid_dict = {key: str(uuid.uuid4()) for key in nans['place_id'].unique()}
nans['Unq_ID'] = nans.place_id.map(uuid_dict)

In [23]:
#jrs

__lookup_dict = ({}, {})
    


jrs.apply(lambda x: create_meta_names(x['Owners'], x['place_id']), axis=1)


uuid_dict = {key: str(uuid.uuid4()) for key in jrs['comb_addr'].unique()}
jrs['Unq_ID'] = jrs.comb_addr.map(uuid_dict)

def update_match(x):
    global jrs
    a = jrs["Owners"].loc[jrs['Unq_ID'] == x].index.values
    p_list=[]
    for i in a:
        if i not in p_list:
            uid = uuid.uuid4()
            matches = match_name(jrs['Owners'][i])
            tmp = []
            for m in matches:
                y = common_members(a, m[1])
                if y != None:
                    for z in y:
                        if z not in tmp:
                            tmp.append(z)
            for g in tmp:
                jrs.at[g, 'Unq_ID'] = str(uid)
            p_list.extend(tmp)
        else:
            continue

x = jrs['Unq_ID'].unique()
f = lambda x: update_match(x)
[f(xi) for xi in x]

<ipython-input-23-f4c4a6a1e089>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jrs['Unq_ID'] = jrs.comb_addr.map(uuid_dict)


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [24]:
final_table=pd.concat([other, family, nans, jrs])

In [42]:
unq_count = final_table.groupby(['Unq_ID']).count().OBJECTID_x.to_dict()
final_table['Total_Parcels_Owned'] = final_table.Unq_ID.map(unq_count)

In [47]:
final_table.to_csv(os.path.dirname(os.getcwd())+'\\OUTPUTS\\INTERMEDIATE\\state_temp.csv')

KeyboardInterrupt: 

In [ ]:
print('Name Records Matched')